In [1]:
import sys
import os
import numpy as np 
from keras.utils import to_categorical
from validation import compute_f1
from keras.models import Model
from keras.layers import TimeDistributed,Conv1D,Dense,Embedding,Input,Dropout,LSTM,Bidirectional,MaxPooling1D,Flatten,concatenate
# from prepro import readfile,createBatches,createMatrices,iterate_minibatches,addCharInformatioin,padding
from keras.utils import plot_model,Progbar
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import RandomUniform
from keras.optimizers import Adam
import keras.backend as K
from sklearn.metrics import f1_score
from keras.callbacks import ModelCheckpoint, Callback
from numpy import newaxis
import sklearn
import subprocess
import fastText
import pickle

/home/gwiedemann/miniconda3/envs/kerasenv/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [55]:
def getCasing(word, caseLookup):   
    casing = 'other'
    
    numDigits = 0
    for char in word:
        if char.isdigit():
            numDigits += 1
            
    digitFraction = numDigits / float(len(word))
    
    if word.isdigit(): #Is a digit
        casing = 'numeric'
    elif digitFraction > 0.5:
        casing = 'mainly_numeric'
    elif word.islower(): #All lower case
        casing = 'allLower'
    elif word.isupper(): #All upper case
        casing = 'allUpper'
    elif word[0].isupper(): #is a title, initial char upper, then all lower
        casing = 'initialUpper'
    elif numDigits > 0:
        casing = 'contains_digit'
    
   
    return caseLookup[casing]

In [3]:
epochs = 80
trainable = True

In [4]:
from keras_contrib.layers import CRF


In [6]:
# changing all deriv and part to misc. with BIO
def modify_labels(dataset):
    bad_labels = ['I-PERderiv','I-OTHpart','B-ORGderiv', 'I-OTH','B-OTHpart','B-LOCderiv','I-LOCderiv','I-OTHderiv','B-PERderiv','B-OTHderiv','B-PERpart','I-PERpart','I-LOCpart','B-LOCpart','I-ORGpart','I-ORGderiv','B-ORGpart','B-OTH']
    for sentence in dataset:
        for word in sentence:
            label = word[1]
            if label in bad_labels:
                first_char = label[0]
                if first_char == 'B' :
                    word[1] = 'B-MISC'
                else:
                    word[1] = 'I-MISC'
    return dataset
                

In [7]:
def get_sentences_germeval(path):
    sentences = []
    with open(path, 'r', encoding = 'UTF-8') as f:
        sentence = []
        for line in f:
            
            line = line.strip()
            
            # append sentence
            if len(line) == 0:
                if len(sentence):
                    sentences.append(sentence)
                sentence = []
                continue
            
            # get sentence tokens
            splits = line.split()
            if splits[0] == '#':
                continue
            temp = [splits[1],splits[2]]
            sentence.append(temp)
        
        # append last
        if len(sentence):
            sentences.append(sentence)    
    return sentences

In [8]:
# preproecessing data from Conll
def get_sentences_conll(filename):
    '''
        -DOCSTART- -X- -X- O

    EU NNP B-NP B-ORG
    rejects VBZ B-VP O
    German JJ B-NP B-MISC
    call NN I-NP O
    to TO B-VP O
    boycott VB I-VP O
    British JJ B-NP B-MISC
    lamb NN I-NP O
    . . O O
    
    '''
    
    '''
    read file
    return format :
    [ ['EU', 'B-ORG'], ['rejects', 'O'], ['German', 'B-MISC'], ['call', 'O'], ['to', 'O'], ['boycott', 'O'], ['British', 'B-MISC'], ['lamb', 'O'], ['.', 'O'] ]
    '''
    f = open(filename,'rb')
    sentences = []
    sentence = []
    for line in f:
        splits = line.split()
        try:
            word=splits[0].decode()
            if word=='-DOCSTART-':
                continue
            label=splits[-1].decode()
            temp=[word,label]
            sentence.append(temp)
        except Exception as e:
            if len(sentence)!=0:
                sentences.append(sentence)
                sentence=[]
    return sentences



In [9]:
trainSentences = get_sentences_germeval('../data/GermEVAL/NER-de-train.tsv')
devSentences = get_sentences_germeval('../data/GermEVAL/NER-de-dev.tsv')
testSentences = get_sentences_germeval('../data/GermEVAL/NER-de-test.tsv')

# trainSentences = get_sentences('../data/CONLL/deu/deu_utf.train')
# devSentences = get_sentences('../data/CONLL/deu/deu_utf.testa')
# testSentences = get_sentences('../data/CONLL/deu/deu_utf.testb')

print(len(trainSentences))
print(len(devSentences))
print(len(testSentences))


24000
2200
5100


In [10]:
print(testSentences[0])

[['1951', 'O'], ['bis', 'O'], ['1953', 'O'], ['wurde', 'O'], ['der', 'O'], ['nördliche', 'O'], ['Teil', 'O'], ['als', 'O'], ['Jugendburg', 'O'], ['des', 'O'], ['Kolpingwerkes', 'B-OTH'], ['gebaut', 'O'], ['.', 'O']]


In [11]:
labelSet = set()
characters= set()

for dataset in [trainSentences, devSentences, testSentences]:
    for sentence in dataset:
        for word, label in sentence:
            for char in word:
                characters.add(char)
            labelSet.add(label)

In [12]:
print(len(labelSet))

25


In [13]:
# :: Create a mapping for the labels ::
label2Idx = {}
for label in labelSet:
    label2Idx[label] = len(label2Idx)

In [14]:
print(label2Idx)

{'O': 0, 'I-LOCderiv': 1, 'B-PER': 2, 'B-OTHderiv': 4, 'I-PERderiv': 5, 'B-ORGpart': 3, 'I-PER': 7, 'I-OTHpart': 6, 'I-OTH': 8, 'B-LOC': 9, 'B-ORGderiv': 10, 'I-LOC': 18, 'B-LOCpart': 17, 'I-PERpart': 20, 'B-PERderiv': 11, 'I-OTHderiv': 12, 'B-PERpart': 13, 'I-ORG': 19, 'B-OTHpart': 14, 'B-LOCderiv': 15, 'I-ORGpart': 21, 'I-LOCpart': 24, 'I-ORGderiv': 22, 'B-ORG': 23, 'B-OTH': 16}


In [15]:
# :: Hard coded case lookup ::
case2Idx = {'numeric': 0, 'allLower':1, 'allUpper':2, 'initialUpper':3, 'other':4, 'mainly_numeric':5, 'contains_digit': 6, 'PADDING_TOKEN':7}
caseEmbeddings = np.identity(len(case2Idx), dtype='float32')

In [16]:
print(caseEmbeddings)
print(case2Idx)

[[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]
{'allLower': 1, 'numeric': 0, 'initialUpper': 3, 'PADDING_TOKEN': 7, 'contains_digit': 6, 'allUpper': 2, 'mainly_numeric': 5, 'other': 4}


In [17]:
print(type(trainSentences))

<class 'list'>


In [18]:
print(trainSentences[0])

[['Schartau', 'B-PER'], ['sagte', 'O'], ['dem', 'O'], ['"', 'O'], ['Tagesspiegel', 'B-ORG'], ['"', 'O'], ['vom', 'O'], ['Freitag', 'O'], [',', 'O'], ['Fischer', 'B-PER'], ['sei', 'O'], ['"', 'O'], ['in', 'O'], ['einer', 'O'], ['Weise', 'O'], ['aufgetreten', 'O'], [',', 'O'], ['die', 'O'], ['alles', 'O'], ['andere', 'O'], ['als', 'O'], ['überzeugend', 'O'], ['war', 'O'], ['"', 'O'], ['.', 'O']]


In [19]:
char2Idx={}
for char in characters:
    char2Idx[char] = len(char2Idx)
char2Idx['UNKNOWN'] = len(char2Idx)
print(char2Idx)

{'術': 0, '8': 1, 'オ': 2, '“': 3, 'ş': 219, ':': 4, 'Â': 5, 'έ': 6, 'П': 218, 'ا': 203, '‘': 7, 'ɨ': 8, 'Π': 9, '士': 117, 'д': 10, '柯': 16, '→': 14, '’': 13, 'ن': 15, '太': 17, 'п': 212, '南': 191, 'î': 286, '_': 19, '傳': 21, '-': 169, 'ς': 296, '▪': 22, 'Т': 320, 'ό': 24, 'U': 25, 'ı': 26, '!': 27, 'а': 28, 'e': 29, 'Ş': 38, 'ô': 35, '†': 34, 'æ': 37, 'N': 325, 'ї': 40, 'ğ': 41, 'з': 42, 'υ': 231, 'И': 208, 'ж': 43, 'л': 44, '6': 45, 'L': 47, 'È': 49, 'Ц': 53, '동': 54, '貴': 55, 'ū': 56, 'ī': 58, 'Е': 59, 'y': 60, 'λ': 61, 'Î': 275, 'r': 63, '²': 64, 'ế': 65, '[': 67, 'q': 68, '\u200e': 70, 'х': 71, '4': 73, '}': 76, 'ν': 282, '博': 249, '別': 77, '3': 234, 'ي': 62, 'أ': 78, 'k': 79, 'Ø': 80, '´': 82, 'O': 83, '§': 72, '守': 85, 'ـ': 86, '9': 88, '½': 91, '¸': 247, 'ħ': 18, '루': 93, 'ä': 12, 'ř': 95, '‹': 97, '算': 99, 'м': 180, 'K': 101, 'í': 103, 'Č': 305, '≘': 75, 'ā': 276, '\x94': 111, 'a': 108, 'C': 74, '李': 113, 'ŏ': 114, 'x': 115, '鷹': 221, 'ʻ': 183, '造': 116, 'є': 283, '台': 118, 'z': 

In [20]:
print(trainSentences[0]) 

[['Schartau', 'B-PER'], ['sagte', 'O'], ['dem', 'O'], ['"', 'O'], ['Tagesspiegel', 'B-ORG'], ['"', 'O'], ['vom', 'O'], ['Freitag', 'O'], [',', 'O'], ['Fischer', 'B-PER'], ['sei', 'O'], ['"', 'O'], ['in', 'O'], ['einer', 'O'], ['Weise', 'O'], ['aufgetreten', 'O'], [',', 'O'], ['die', 'O'], ['alles', 'O'], ['andere', 'O'], ['als', 'O'], ['überzeugend', 'O'], ['war', 'O'], ['"', 'O'], ['.', 'O']]


In [21]:
ft = fastText.load_model("../embeddings/wiki.de.bin")
# ft = fastText.load_model("../embeddings/cc.de.300.bin")

nb_embedding_dims = ft.get_dimension()

In [22]:
print(nb_embedding_dims)
print(len(trainSentences[0]))

300
25


In [23]:
print(trainSentences[0])

[['Schartau', 'B-PER'], ['sagte', 'O'], ['dem', 'O'], ['"', 'O'], ['Tagesspiegel', 'B-ORG'], ['"', 'O'], ['vom', 'O'], ['Freitag', 'O'], [',', 'O'], ['Fischer', 'B-PER'], ['sei', 'O'], ['"', 'O'], ['in', 'O'], ['einer', 'O'], ['Weise', 'O'], ['aufgetreten', 'O'], [',', 'O'], ['die', 'O'], ['alles', 'O'], ['andere', 'O'], ['als', 'O'], ['überzeugend', 'O'], ['war', 'O'], ['"', 'O'], ['.', 'O']]


In [24]:
def createBatches(dataset):
    l = []
    for i in dataset:
        l.append(len(i))
    l = set(l)
    print(len(l))
    batches = []
    batch_len = []
    z = 0
    for i in l:
        temp = []
        for batch in dataset:
            if len(batch) == i:
                temp.append(batch)
                z += 1
        batches.append(temp)
#         batch_len.append(z)
    return batches

In [25]:
train_batches = createBatches(trainSentences)
dev_batches = createBatches(devSentences)
test_batches = createBatches(testSentences)

51
45
47


In [26]:
batches = train_batches
print(len(batches))
print(len(batches[0]))
print(batches[0])


51
3
[[['Alles', 'O'], ['richtig', 'O'], ['.', 'O']], [['Farben', 'O'], ['eingeführt', 'O'], ['.', 'O']], [['Material', 'O'], ['gewinnen', 'O'], ['.', 'O']]]


In [62]:
nb_char_embedding_dims = 52

def generator(batches: 'list of training/dev sentences- batches already created'):
    global line_number
    
    while True:
        for batch in batches:
            word_embeddings = []
            case_embeddings = []
            char_embeddings = []

            output_labels = []
            for index in range(len(batch)): # batches made according to the size of the sentences. len(batch) gives the size of current batch
                sentence = batch[index]
    #             print(sentence)
                temp_casing = []
                temp_char=[]
                temp_word=[]
                temp_output=[]
                for word in sentence:
                    word, label = word
                    casing =getCasing(word, case2Idx)
                    temp_casing.append(casing)
                    temp_char2=[]
                    for char in word:
                        if char in char2Idx.keys():
                            temp_char2.append(char2Idx[char])
                        else:
                            temp_char2.append(char2Idx['UNKNOWN']) # To incorporate the words which are not in the vocab
                    temp_char2 = np.array(temp_char2)
                    temp_char.append(temp_char2)
                    word_vector = ft.get_word_vector(word.lower())
                    # word_vector = ft.get_word_vector(word)
                    temp_word.append(word_vector)
                    temp_output.append(label2Idx[label])
                temp_char = pad_sequences(temp_char, nb_char_embedding_dims)
                word_embeddings.append(temp_word)
                case_embeddings.append(temp_casing)
                char_embeddings.append(temp_char)
                temp_output = to_categorical(temp_output, 25)
                output_labels.append(temp_output)
    #             output_labels = to_categorical()
    #             output_labels = np.array(output_labels)
    #             output_labels = output_labels[...,newaxis]

    #             print(np.array(word_embeddings).shape)
    #             print(np.array(case_embeddings).shape)
    #             print(np.array(char_embeddings).shape)
    #             print(output_labels.shape)
    #             print("******************\n\n")
            yield ([np.array(word_embeddings), np.array(case_embeddings), np.array(char_embeddings)], np.array(output_labels))

def get_label_from_categorical(a):
    labels = []
    for label in a:
        label = np.ndarray.tolist(label)
        label = np.argmax(label)
        labels.append(label)
    return(labels)

def predict_batches(batch):
    steps = 0
    true_labels = []
    pred_labels = []
    for input_data, output_data in generator(batch):
        pred_labels_batch = model.predict(input_data)
        for s in pred_labels_batch:
            pred_labels.append(get_label_from_categorical(s))
        for s in output_data:
            true_labels.append(get_label_from_categorical(s))
        steps += 1
        if steps == len(batch):
            break
    return(true_labels, pred_labels)

In [28]:
idx2Label = {v: k for k, v in label2Idx.items()}

In [29]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1


In [63]:
def f1_score(y_true, y_pred):
    true_labels = []
    pred_labels = []
    for s in y_pred:
        pred_labels.append(get_label_from_categorical(s))
    for s in y_true:
        true_labels.append(get_label_from_categorical(s))
    p, r, f = compute_f1(y_pred, y_true, idx2Label)
    return r

def get_model():
    words_input = Input(shape=(None, nb_embedding_dims), dtype='float32', name='words_input')
    casing_input = Input(shape=(None,), dtype='int32', name='casing_input')
    casing = Embedding(output_dim=caseEmbeddings.shape[1], input_dim=caseEmbeddings.shape[0], weights=[caseEmbeddings], trainable=False, name = 'case_embed')(casing_input)
    character_input=Input(shape=(None,52,),name='char_input')
    embed_char_out=TimeDistributed(Embedding(len(char2Idx),32,embeddings_initializer=RandomUniform(minval=-0.5, maxval=0.5)), name='char_embedding')(character_input)
    kernel_sizes = (3, 4, 5)
    conv_blocks = []
    for sz in kernel_sizes:
        conv = TimeDistributed(Conv1D(
                             kernel_size=sz,
                             filters=32,
                             padding="same",
                             activation="relu",
                             strides=1))(embed_char_out)
        conv = TimeDistributed(MaxPooling1D(52))(conv)
        conv = TimeDistributed(Flatten())(conv)
        conv_blocks.append(conv)
    output = concatenate([words_input, casing, conv_blocks[0], conv_blocks[1], conv_blocks[2]])
    output = Bidirectional(LSTM(200, return_sequences=True, dropout=0.50, recurrent_dropout=0.5))(output)
    output = TimeDistributed(Dense(len(label2Idx)))(output)
    crf = CRF(len(label2Idx))
    output = crf(output)
    model = Model(inputs=[words_input, casing_input, character_input], outputs=[output])
    model.compile(loss=crf.loss_function, optimizer='nadam', metrics=[crf.accuracy])
    model.summary()
    return(model)

def get_model_lstm():
    words_input = Input(shape=(None, nb_embedding_dims), dtype='float32', name='words_input')
    casing_input = Input(shape=(None,), dtype='int32', name='casing_input')
    casing = Embedding(output_dim=caseEmbeddings.shape[1], input_dim=caseEmbeddings.shape[0], weights=[caseEmbeddings], trainable=False, name = 'case_embed')(casing_input)
    character_input=Input(shape=(None,nb_char_embedding_dims,),name='char_input')
    embed_char_out=TimeDistributed(Embedding(len(char2Idx),32,embeddings_initializer=RandomUniform(minval=-0.5, maxval=0.5)), name='char_embedding')(character_input)
    char_lstm = TimeDistributed(Bidirectional(LSTM(52)))(embed_char_out)
    output = concatenate([words_input, casing, char_lstm])
    output = Bidirectional(LSTM(200, return_sequences=True, dropout=0.50, recurrent_dropout=0.5))(output)
    output = TimeDistributed(Dense(len(label2Idx)))(output)
    crf = CRF(len(label2Idx))
    output = crf(output)
    model = Model(inputs=[words_input, casing_input, character_input], outputs=[output])
    model.compile(loss=crf.loss_function, optimizer='nadam', metrics=[crf.accuracy])
    model.summary()
    return(model)

class F1History(Callback):
    def on_train_begin(self, logs={}):
        self.acc = []
        self.f1_scores = []
        self.max_f1 = 0

    def on_epoch_end(self, epoch, logs={}):
        self.acc.append(logs.get('val_acc'))
        true_labels, pred_labels = predict_batches(dev_batches)
        pre, rec, f1 = compute_f1(pred_labels, true_labels, idx2Label)
        self.f1_scores.append(f1)
        if epoch > 30 and f1 > self.max_f1:
            print("\nNew maximum F1 score: " + str(f1) + " (before: " + str(self.max_f1) + ") Saving to " + tmp_model_filename)
            self.max_f1 = f1
            model.save(tmp_model_filename)

In [ ]:
tmp_model_filename = 'tmp_generator_NER_lstm_best.h5'
# checkpoint = ModelCheckpoint(tmp_model_filename, verbose=1, save_best_only = True, monitor = 'val_acc')
history = F1History()
model = get_model_lstm()
model.fit_generator(
    generator(train_batches), 
    epochs = 80, steps_per_epoch = len(train_batches), 
    validation_data = generator(dev_batches), validation_steps = len(dev_batches), 
    callbacks = [history]
)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_input (InputLayer)         (None, None, 52)     0                                            
__________________________________________________________________________________________________
casing_input (InputLayer)       (None, None)         0                                            
__________________________________________________________________________________________________
char_embedding (TimeDistributed (None, None, 52, 32) 10528       char_input[0][0]                 
__________________________________________________________________________________________________
words_input (InputLayer)        (None, None, 300)    0                                            
__________________________________________________________________________________________________
case_embed

In [60]:
print(history.acc)
print(history.f1)

[]


AttributeError: 'F1History' object has no attribute 'f1'

In [ ]:
model.load_weights(tmp_model_filename)

In [ ]:
true_labels, pred_labels = predict_batches(test_batches)
print(compute_f1(pred_labels, true_labels, idx2Label))

# Experiments

In [61]:
f = open('results_lstm.txt', 'w')
for run_i in range(10):
    print("Run " + str(run_i))
    
    tmp_model_filename = 'tmp_generator_NER_lstm_best.' + str(run_i) + '.h5'
    # tmp_model_filename = 'tmp_generator_NER_best.h5'
    # checkpoint = ModelCheckpoint(tmp_model_filename, verbose=1, save_best_only = True, monitor = 'val_acc')
    history = F1History()
    
    model = get_model()
    model.fit_generator(
        generator(train_batches), 
        epochs = 80, steps_per_epoch = len(train_batches), 
        validation_data = generator(dev_batches), validation_steps = len(dev_batches), 
        callbacks = [history]
    )
    
    model.load_weights(tmp_model_filename)
    true_labels, pred_labels = predict_batches(test_batches)
    
    pre, rec, f1 = compute_f1(pred_labels, true_labels, idx2Label)
    f.write(str(run_i) + "\t" + str(pre) + "\t" + str(rec) +  "\t" + str(f1))
    f.write("\n")
    f.flush()
f.close()

Run 0
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_input (InputLayer)         (None, None, 100)    0                                            
__________________________________________________________________________________________________
char_embedding (TimeDistributed (None, None, 100, 32 10528       char_input[0][0]                 
__________________________________________________________________________________________________
time_distributed_111 (TimeDistr (None, None, 100, 32 3104        char_embedding[0][0]             
__________________________________________________________________________________________________
time_distributed_114 (TimeDistr (None, None, 100, 32 4128        char_embedding[0][0]             
__________________________________________________________________________________________________
time

51/51 [==============================] - 23s 455ms/step - loss: -8.6236e-04 - acc: 0.9753 - val_loss: 0.0092 - val_acc: 0.9709
Epoch 30/80
51/51 [==============================] - 24s 463ms/step - loss: -0.0065 - acc: 0.9757 - val_loss: 0.0023 - val_acc: 0.9706
Epoch 31/80
51/51 [==============================] - 23s 447ms/step - loss: -0.0130 - acc: 0.9766 - val_loss: -0.0013 - val_acc: 0.9708
Epoch 32/80
50/51 [============================>.] - ETA: 0s - loss: -0.0181 - acc: 0.9762
New maximum F1 score: 0.7691417550626809 (before: 0) Saving to tmp_generator_NER_lstm_best.0.h5
51/51 [==============================] - 28s 546ms/step - loss: -0.0179 - acc: 0.9765 - val_loss: -0.0104 - val_acc: 0.9708
Epoch 33/80
50/51 [============================>.] - ETA: 0s - loss: -0.0249 - acc: 0.9772
New maximum F1 score: 0.7738974256635401 (before: 0.7691417550626809) Saving to tmp_generator_NER_lstm_best.0.h5
51/51 [==============================] - 24s 471ms/step - loss: -0.0247 - acc: 0.9775 -

51/51 [==============================] - 25s 481ms/step - loss: -0.2163 - acc: 0.9870 - val_loss: -0.1831 - val_acc: 0.9739
Epoch 73/80
50/51 [============================>.] - ETA: 0s - loss: -0.2239 - acc: 0.9871
New maximum F1 score: 0.8092552984639315 (before: 0.8027000964320153) Saving to tmp_generator_NER_lstm_best.0.h5
51/51 [==============================] - 24s 479ms/step - loss: -0.2211 - acc: 0.9873 - val_loss: -0.1976 - val_acc: 0.9751
Epoch 74/80
51/51 [==============================] - 24s 463ms/step - loss: -0.2258 - acc: 0.9874 - val_loss: -0.1978 - val_acc: 0.9737
Epoch 75/80
51/51 [==============================] - 24s 466ms/step - loss: -0.2303 - acc: 0.9875 - val_loss: -0.2044 - val_acc: 0.9751
Epoch 76/80
51/51 [==============================] - 24s 464ms/step - loss: -0.2356 - acc: 0.9880 - val_loss: -0.2062 - val_acc: 0.9733
Epoch 77/80
51/51 [==============================] - 24s 463ms/step - loss: -0.2397 - acc: 0.9879 - val_loss: -0.2130 - val_acc: 0.9737
Epoc

51/51 [==============================] - 24s 467ms/step - loss: 0.0697 - acc: 0.9687 - val_loss: 0.0866 - val_acc: 0.9630
Epoch 19/80
51/51 [==============================] - 23s 451ms/step - loss: 0.0637 - acc: 0.9699 - val_loss: 0.1127 - val_acc: 0.9619
Epoch 20/80
51/51 [==============================] - 23s 458ms/step - loss: 0.0586 - acc: 0.9704 - val_loss: 0.0785 - val_acc: 0.9645
Epoch 21/80
51/51 [==============================] - 23s 448ms/step - loss: 0.0506 - acc: 0.9701 - val_loss: 0.0538 - val_acc: 0.9680
Epoch 22/80
51/51 [==============================] - 24s 465ms/step - loss: 0.0441 - acc: 0.9710 - val_loss: 0.0785 - val_acc: 0.9649
Epoch 23/80
51/51 [==============================] - 24s 468ms/step - loss: 0.0381 - acc: 0.9723 - val_loss: 0.0474 - val_acc: 0.9688
Epoch 24/80
51/51 [==============================] - 23s 455ms/step - loss: 0.0295 - acc: 0.9733 - val_loss: 0.0466 - val_acc: 0.9675
Epoch 25/80
51/51 [==============================] - 23s 458ms/step - loss

51/51 [==============================] - 23s 456ms/step - loss: -0.1626 - acc: 0.9831 - val_loss: -0.1489 - val_acc: 0.9745
Epoch 64/80
51/51 [==============================] - 23s 448ms/step - loss: -0.1689 - acc: 0.9840 - val_loss: -0.1536 - val_acc: 0.9739
Epoch 65/80
50/51 [============================>.] - ETA: 0s - loss: -0.1764 - acc: 0.9842
New maximum F1 score: 0.8051400076716534 (before: 0.8038424591738714) Saving to tmp_generator_NER_lstm_best.1.h5
51/51 [==============================] - 24s 468ms/step - loss: -0.1742 - acc: 0.9844 - val_loss: -0.1596 - val_acc: 0.9745
Epoch 66/80
50/51 [============================>.] - ETA: 0s - loss: -0.1820 - acc: 0.9846
New maximum F1 score: 0.8086325439266616 (before: 0.8051400076716534) Saving to tmp_generator_NER_lstm_best.1.h5
51/51 [==============================] - 24s 464ms/step - loss: -0.1797 - acc: 0.9848 - val_loss: -0.1644 - val_acc: 0.9751
Epoch 67/80
51/51 [==============================] - 23s 458ms/step - loss: -0.1853 

...

51/51 [==============================] - 24s 471ms/step - loss: -0.1190 - acc: 0.9819 - val_loss: -0.1072 - val_acc: 0.9735
Epoch 49/80
51/51 [==============================] - 23s 458ms/step - loss: -0.1244 - acc: 0.9821 - val_loss: -0.1135 - val_acc: 0.9728
Epoch 50/80
51/51 [==============================] - 35s 680ms/step - loss: -0.1292 - acc: 0.9826 - val_loss: -0.1042 - val_acc: 0.9728
Epoch 51/80
51/51 [==============================] - 37s 727ms/step - loss: -0.1339 - acc: 0.9825 - val_loss: -0.1183 - val_acc: 0.9737
Epoch 52/80
51/51 [==============================] - 25s 498ms/step - loss: -0.1394 - acc: 0.9830 - val_loss: -0.1220 - val_acc: 0.9741
Epoch 53/80
50/51 [============================>.] - ETA: 0s - loss: -0.1460 - acc: 0.9832
New maximum F1 score: 0.7960867063111452 (before: 0.7944465869649056) Saving to tmp_generator_NER_lstm_best.4.h5
51/51 [==============================] - 26s 508ms/step - loss: -0.1442 - acc: 0.9834 - val_loss: -0.1308 - val_acc: 0.9741
Epoc

Epoch 1/80
51/51 [==============================] - 48s 950ms/step - loss: 0.5989 - acc: 0.8822 - val_loss: 1.5309 - val_acc: 0.3256
Epoch 2/80
51/51 [==============================] - 29s 565ms/step - loss: 0.3248 - acc: 0.9221 - val_loss: 0.7904 - val_acc: 0.8009
Epoch 3/80
51/51 [==============================] - 26s 508ms/step - loss: 0.2565 - acc: 0.9352 - val_loss: 0.9733 - val_acc: 0.5600
Epoch 4/80
51/51 [==============================] - 25s 495ms/step - loss: 0.2500 - acc: 0.9309 - val_loss: 0.3830 - val_acc: 0.9363
Epoch 5/80
51/51 [==============================] - 36s 713ms/step - loss: 0.2032 - acc: 0.9428 - val_loss: 0.5214 - val_acc: 0.8422
Epoch 6/80
51/51 [==============================] - 30s 586ms/step - loss: 0.1756 - acc: 0.9472 - val_loss: 0.3797 - val_acc: 0.8777
Epoch 7/80
51/51 [==============================] - 27s 532ms/step - loss: 0.1593 - acc: 0.9503 - val_loss: 0.3562 - val_acc: 0.8683
Epoch 8/80
51/51 [==============================] - 30s 590ms/step - 

50/51 [============================>.] - ETA: 0s - loss: -0.1246 - acc: 0.9823
New maximum F1 score: 0.793854531310774 (before: 0.7925262748151032) Saving to tmp_generator_NER_lstm_best.5.h5
51/51 [==============================] - 36s 713ms/step - loss: -0.1231 - acc: 0.9826 - val_loss: -0.1112 - val_acc: 0.9733
Epoch 50/80
50/51 [============================>.] - ETA: 0s - loss: -0.1293 - acc: 0.9822
New maximum F1 score: 0.7964809384164224 (before: 0.793854531310774) Saving to tmp_generator_NER_lstm_best.5.h5
51/51 [==============================] - 31s 609ms/step - loss: -0.1277 - acc: 0.9824 - val_loss: -0.1131 - val_acc: 0.9743
Epoch 51/80
51/51 [==============================] - 24s 468ms/step - loss: -0.1336 - acc: 0.9828 - val_loss: -0.1198 - val_acc: 0.9735
Epoch 52/80
51/51 [==============================] - 29s 566ms/step - loss: -0.1377 - acc: 0.9828 - val_loss: -0.1191 - val_acc: 0.9736
Epoch 53/80
50/51 [============================>.] - ETA: 0s - loss: -0.1445 - acc: 0.

Epoch 1/80
51/51 [==============================] - 44s 859ms/step - loss: 0.5944 - acc: 0.8808 - val_loss: 0.9106 - val_acc: 0.7010
Epoch 2/80
51/51 [==============================] - 30s 594ms/step - loss: 0.3208 - acc: 0.9223 - val_loss: 1.1161 - val_acc: 0.6027
Epoch 3/80
51/51 [==============================] - 25s 491ms/step - loss: 0.2659 - acc: 0.9353 - val_loss: 1.0630 - val_acc: 0.5161
Epoch 4/80
51/51 [==============================] - 25s 494ms/step - loss: 0.2285 - acc: 0.9378 - val_loss: 0.9693 - val_acc: 0.5677
Epoch 5/80
51/51 [==============================] - 29s 563ms/step - loss: 0.2132 - acc: 0.9402 - val_loss: 0.7053 - val_acc: 0.7612
Epoch 6/80
51/51 [==============================] - 25s 486ms/step - loss: 0.1824 - acc: 0.9473 - val_loss: 0.3152 - val_acc: 0.9131
Epoch 7/80
51/51 [==============================] - 24s 480ms/step - loss: 0.1566 - acc: 0.9527 - val_loss: 0.3141 - val_acc: 0.8967
Epoch 8/80
51/51 [==============================] - 25s 482ms/step - 

KeyboardInterrupt: 

# Save final model

In [ ]:
import shutil, json
# copy file for best run
shutil.copyfile('tmp_generator_NER_best.0.h5', 'final_model_germeval.h5')
with open("final_model_germeval.indexes", "w") as f:
    json.dump([idx2Label, label2Idx, char2Idx, case2Idx], f)